faiss事前準備:

miniconda_python(3.10版本):

https://docs.conda.io/projects/miniconda/en/latest/miniconda-other-installer-links.html

裝完打開anaconda prompt:

做一個環境```conda create -n VScode-env-3.10.13 python=3.10.13```

做完啟動```conda activate VScode-env-3.10.13```

安裝faiss插件

```conda install -c pytorch faiss-cpu=1.7.4 mkl=2021 blas=1.0=mkl```

```conda install -c pytorch/label/nightly faiss-cpu```

關掉prompt 重新開啟VScode 右上角kernel 選擇another kernel->python env->VScode-env-3.10.13

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:```pip install openai```

沒裝tabulate使用:```pip install tabulate```

沒裝pandas:```pip install pandas```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

In [25]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# input在此輸入

In [26]:
input = "hey flora! Im having some issue using Instagram! can you teach me how to add friend?"

先準備faiss

讀embeddings資料

並把dataframe轉為numpy

In [27]:
import pandas as pd
import numpy as np

csv_file = 'data\Qembeddings.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,-0.022805,-0.003135,-0.013681,-0.036727,-0.044497,0.029007,-0.024501,-0.041485,-0.005907,-0.009333,...,-0.001190,-0.021287,0.009650,-0.010143,-0.028197,0.025374,-0.010055,-0.003249,-0.007062,-0.028197
1,-0.016840,-0.011369,-0.003460,-0.017197,-0.024066,0.012614,-0.030361,-0.040702,0.012908,-0.003005,...,0.013699,-0.041800,0.021296,-0.006463,-0.004542,-0.020491,-0.033731,0.010705,0.003454,-0.024437
2,-0.020366,-0.011115,0.016573,-0.021214,-0.017512,-0.002729,-0.011475,-0.030523,0.020134,-0.016496,...,0.023915,-0.022757,0.027797,-0.001901,-0.010009,0.003963,-0.034329,-0.025637,0.005378,-0.002792
3,-0.029621,-0.031300,-0.004523,-0.022191,-0.040698,0.019835,-0.013771,-0.023770,0.014309,-0.008721,...,0.015738,-0.005817,0.005927,0.000405,-0.026338,0.000488,-0.020086,-0.000731,0.001592,-0.020474
4,-0.014236,-0.011760,-0.014080,-0.002142,-0.036923,0.014444,-0.035155,-0.033881,0.009361,-0.001141,...,0.014509,-0.018397,0.017265,-0.009296,-0.004875,-0.010771,-0.011649,0.004901,0.003660,-0.035259


確認內容

In [28]:
print(data.shape[0])
print(type(data))
print(data.dtype)
print(data[0][0])

669
<class 'numpy.ndarray'>
float64
-0.0228053741157054


寫入維度 並把data加入faiss的index

In [29]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

做問題的embedding

In [30]:
response = client.embeddings.create(
    input=input,
    model="text-embedding-ada-002"
)
question = np.array(response.data[0].embedding, ndmin=2)
print(question)

[[-0.0192867   0.00081752 -0.0193528  ... -0.00241414  0.00399548
  -0.05195116]]


TOP-k的設定並查找問題

In [31]:
k = 3

distances, indices = index.search(question, k)

拿答案

In [32]:
csv_question = 'data\manual.csv'
qdf = pd.read_csv(csv_question)
qarray = qdf.to_numpy()

看答案和距離

In [33]:
answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
print(answer)

[(array(["SoI'm trying to add Facebook friends on flora but I just don't get the option , can you please assist ?",
       "Currently, we are dropping the Facebook option due to some security reasons. If you want to add your friend, an alternative way is to add his/her account to your contact info and use the 'Import from Contact' function."],
      dtype=object), 0.26772311329841614), (array(['Are there any plans to make a Flora instagram profile? because I think it would help boost your popularity',
       'Currently we do not have an official Instagram page. We are considering opening an Instagram page and might plan to open one in one day. '],
      dtype=object), 0.31527459621429443), (array(["I'm a flora user, and I really like the new version of flora! However, is there a way to remove certain friends? I think some of my friends were imported from Facebook since my flora account is linked to Facebook. But I would like to keep only some close friends in flora.",
       'To remove

# Prompt內容在此

In [34]:
history = ""
for i in range(0,k):
    history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
prompt = "Here is "+str(k)+" history questions and answers:\n"+history+"Here is my question:\n"+input
print(prompt)

Here is 3 history questions and answers:
1. SoI'm trying to add Facebook friends on flora but I just don't get the option , can you please assist ?
Currently, we are dropping the Facebook option due to some security reasons. If you want to add your friend, an alternative way is to add his/her account to your contact info and use the 'Import from Contact' function.
2. Are there any plans to make a Flora instagram profile? because I think it would help boost your popularity
Currently we do not have an official Instagram page. We are considering opening an Instagram page and might plan to open one in one day. 
3. I'm a flora user, and I really like the new version of flora! However, is there a way to remove certain friends? I think some of my friends were imported from Facebook since my flora account is linked to Facebook. But I would like to keep only some close friends in flora.
To remove a friend, you could swipe him/her to the left on the Friend Page, and click it again for confirmati

In [35]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are customer service from Flora.You can only answer questions related to Flora. when answering any question, you must always answer base on history's questions. if history's questions are not similar to user's question, you must not answer."},
    {"role": "user", "content": prompt}
  ]
)

In [36]:
print(completion.choices[0].message.content)

Currently we do not have an official Instagram page. We are considering opening an Instagram page and might plan to open one in one day.


創assistant,thread,message!

from tabulate import tabulate

assistant = client.beta.assistants.create(
    name="Flora Customer Service",
    instructions="You are customer service from Flora.You can only answer questions related to Flora. when answering any question, you must always answer base on history's questions. if history's questions are not similar to user's question, you must not answer.",
    model="gpt-3.5-turbo-1106"
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= prompt
)


沒裝tabulate很醜

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

print(tabulate(assistant,headers=['assistant','content'],tablefmt="simple_outline"))

print(tabulate(thread,headers=['thread','content'],tablefmt="simple_outline"))

print(tabulate(message,headers=['message','content'],tablefmt="simple_outline"))

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

跑完後打開查看history messages!

messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

最後一個訊息顯示

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages.data[0].content[0].text.value)